In [4]:
####exposure####
import basefunc
import numpy as np
import os

'''01 absolute exposure'''
def absolute_exposure(data):
    '''
    data.shape[0]: change in return period
    data.shape[1]: lat
    data.shape[2]: lon
    '''
    data[0,:,:][data[1,:,:]>-0.25]=0
    data[0,:,:][data[1,:,:]==-1]=0
    return data[0,:,:]

#by assets type
types=['MV','LV']
SCENARIO_YEARS = {
    
    'ssp126': [(2030, 2060), (2070, 2100)],
    'ssp245': [(2030, 2060), (2070, 2100)],
    'ssp370': [(2030, 2060), (2070, 2100)],
    'ssp585': [(2030, 2060), (2070, 2100)],
}

ref_tif='./data/ref.tif'
for name in types:
    for scen, ranges in SCENARIO_YEARS.items():
        for time_low,time_up in ranges:
            data=basefunc.getRaster('./results_for_rt_change/f_normal/'+name+'_under_'+scen+'_'+str(time_low)+str(time_up)+'_mme.tif')
            a_exposure = absolute_exposure(data)
            
            output_path='./results_for_exposure/f_normal/'+name+'_AE_under_'+scen+'_'+str(time_low)+str(time_up)+'_mme.tif'
            basefunc.array2Raster(a_exposure,ref_tif,output_path)    
#all types
for scen, ranges in SCENARIO_YEARS.items():
    for time_low,time_up in ranges:
        a_exposure=np.full((600,1440),0.0)
        for name in types:  
            data=basefunc.getRaster('./results_for_rt_change/f_normal/'+name+'_under_'+scen+'_'+str(time_low)+str(time_up)+'_mme.tif')
            a_exposure = absolute_exposure(data) + a_exposure
        
        output_path='./results_for_exposure/f_normal/AE_under_'+scen+'_'+str(time_low)+str(time_up)+'_mme.tif'
        basefunc.array2Raster(a_exposure,ref_tif,output_path)  


In [5]:
'''02 relative exposure'''
def relative_exposure(exposed_length,length):
    exposure=np.sum(exposed_length,axis=0)
    total_length=np.sum(length,axis=0)
    return exposure/total_length

#by types
types=['MV','LV']
SCENARIO_YEARS = {
    
    'ssp126': [(2030, 2060), (2070, 2100)],
    'ssp245': [(2030, 2060), (2070, 2100)],
    'ssp370': [(2030, 2060), (2070, 2100)],
    'ssp585': [(2030, 2060), (2070, 2100)],
}

ref_tif='./data/ref.tif'
for name in types:
    for scen, ranges in SCENARIO_YEARS.items():
        for time_low,time_up in ranges:
            exposed_length = basefunc.getRaster('./results_for_exposure/f_normal/'+name+'_AE_under_'+scen+'_'+str(time_low)+str(time_up)+'_mme.tif')
            
            length = basefunc.getRaster(f'./{name}.tif')
    
            r_exposure = exposed_length/length
            
            output_path='./results_for_exposure/f_normal/'+name+'_RE_under_'+scen+'_'+str(time_low)+str(time_up)+'_mme.tif'
            basefunc.array2Raster(r_exposure,ref_tif,output_path)          


for scen, ranges in SCENARIO_YEARS.items():
    for time_low,time_up in ranges:
        if 'exposed_length' in vars():
            del exposed_length
        if 'length' in vars():
            del length            
        for name in types:  
            a_exposure=basefunc.getRaster('./results_for_exposure/f_normal/'+name+'_AE_under_'+scen+'_'+str(time_low)+str(time_up)+'_mme.tif')
            leng=basefunc.getRaster('./results_for_rt_change/f_normal/'+name+'_under_'+scen+'_'+str(time_low)+str(time_up)+'_mme.tif')[0,:,:]
            rt_change=basefunc.getRaster('./results_for_rt_change/f_normal/'+name+'_under_'+scen+'_'+str(time_low)+str(time_up)+'_mme.tif')[1,:,:]
            leng[rt_change==-1]=0
            
            a_exposure=a_exposure.reshape(1,600,1440)
            leng=leng.reshape(1,600,1440)
            try:
                exposed_length=np.concatenate((exposed_length,a_exposure),axis=0)
                length=np.concatenate((length,leng),axis=0)
            except:
                exposed_length=a_exposure
                length=leng
        
        r_exposure = relative_exposure(exposed_length,length)
    
        output_path='./results_for_exposure/f_normal/RE_under_'+scen+'_'+str(time_low)+str(time_up)+'_mme.tif'
        basefunc.array2Raster(r_exposure,ref_tif,output_path)   

C:\Users\Administrator\AppData\Local\Temp\ipykernel_18956\1261032536.py:26: RuntimeWarning: invalid value encountered in divide
  r_exposure = exposed_length/length
C:\Users\Administrator\AppData\Local\Temp\ipykernel_18956\1261032536.py:5: RuntimeWarning: invalid value encountered in divide
  return exposure/total_length
